# Field monitoring in near-real time

## Background

crop


![Image of Yaktocat](https://wales.livingearth.online/wp-content/uploads/2022/01/croptype.png)

                         Crop field in Pembrokshire. Photograph: Clive Hurford

## Description

This notebook demonstrates how crops and their growth stages can be monitored, at parcel scale, in Near-Real Time using Sentinel-1 Synthetic Aperture Radar (SAR) satellite sensor, within the newly developed Wales Open Data Cube environment.

Recently Planet has started to provide daily satellite data to Wales, which could be very useful for crop monitoring. However, as any optical sensor, Planet imagery is impeded by cloud cover, as well as winter lack of illumination.

In Wales, the period of greatest cloud cover is from October to the end of April, which coincides with the beginning of the growing season for several crops. Moreover, cloud cover remains high during the whole year and as a consequence, through the major crop growth periods. In these conditions, even Planet, with its daily provision capacity, cannot provide near-real time data for monitoring crops during the whole growing season. 

Synthetic Aperture Radar (SAR) data have the advantage of operating at wavelengths not impeded by cloud cover or a lack of illumination and can acquire data over a site during day or night time under all weather conditions. Sentinel-1, with its C-SAR instrument, offers reliable repeated wide area monitoring with 10m spatial resolution.

As part of the new digital infrastructure (incl. Open Data Cube technology), as soon as new Sentinel-1 scenes are made available by Copernicus, data are processed to Analysis Ready Data format and made available on EODataDown and within the Wales Open Data Cube, thus allowing monitoring in Near-Real Time.

This notebook uses the Near-Real Time Analysis Ready Sentinel-1 Data, as well as tools developed by Living Wales, to continuously monitor field plots and their states, incl. beginning of vegetation growth, key growth stages, harvest date.


Topics include: 

1. Query a field plot of interest, by drawing it or selecting a field plot from a shapefile
2. Loading data from datacube
3. Masking data out of the field plot
4. Tracking the evolution of a parcel in near-real time
5. Study case: tracking the growth stages in a Rapeseed plot

### Jupyter Notebooks
#### Running (executing) a cell
Jupyter Notebooks allow code to be separated into sections that can be executed independent of one another.
These sections are called "cells".

Python code is written into individual cells that can be executed by placing the cursor in the cell and typing `Shift-Enter` on the keyboard or selecting the &#9658; "Run" button in the ribbon at the top of the notebook.
These options will run a single cell at a time.

When a cell is run, the cell's content is executed.
Any output produced from running the cell will appear directly below it.

Run the cell below as a test:

In [ ]:
print("I ran a cell!")

#### Cell status
The `[ ]:` symbol to the left of each Code cell describes the state of the cell:

* `[ ]:` means that the cell has not been run yet.
* `[*]:` means that the cell is currently running.
* `[1]:` means that the cell has finished running and was the first cell run. The number indicates the order that the cells were run in.

***

## Getting started

To run this analysis, run all the cells in the notebook starting with the 'Load packages and connect to the datacube' cell and choose an area and period of interest.

### Load packages

Load key Python packages and supporting functions for the analysis, then connect to the datacube. 

In [ ]:
# Import modules

import sys

from time import time as time
import datetime as dt

import geopandas as gpd
from ipyleaflet import GeoData

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

import warnings
warnings.filterwarnings("ignore")

import datacube
from datacube.utils.geometry import Geometry, CRS

sys.path.append("../wales_utils/data_cube_utilities")
from display_tools import draw_select, calendar
from wdc_datahandling import geom_fromdrawn, geopolygon_masking
sys.path.append("../wales_utils/themes_utilities")
from crop import rapeseed_study_case_plot

### Connect to the datacube
Connect to the datacube so we can access Living Wales Analysis Ready Data.

In [ ]:
dc = datacube.Datacube()

### Query a field plot of interest
#### Loading field plots from shapefile

In [ ]:
# Open and read a shapefile containing the boundaries of several field plots
field_plots = gpd.read_file('../Vectors/WDC_workshop.shp')

# Transform the shapefile boundaries into geographic data (and affect a style) 
geo_data = GeoData(geo_dataframe = field_plots.to_crs(epsg=4326),
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 
                          'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Fields')

# map the geographic data
m, option_widget = draw_select(geo_data)
m.add_layer(geo_data)
m

#### Draw/Select a field plot

In this section, we will interact with the above display tool to either select a field plot from the shapefile or draw the boundaries of a plot of interest. 

Unfortunately, currently, there is no ipython library available to click-select. As part of the Living Wales, we have developed an alternative to allow you to select a polygon feature. 

- TO SELECT: use the &#9726; *'draw a rectangle'* symbol in the leftside bar, draw a small polygon **inside** the polygon you would like to select and click on the 'Select' button
- TO DRAW: use the &#11039; *'draw a polygon'* symbol in the leftside bar, draw the boundaries of your chosen field plot and click on the 'Extent' button
- TO DELETE: at any time, you can click on the bin symbol and choose 'Clear All" to delete the polygon(s) you have drawn.


#### Use the drawn/selected field plot as the geopolygon of interest

In [ ]:
# Get the selected option (i.e., Extent or Selection) 
selected_option = option_widget.value

# Transform the drawn boundaries to a geopolygon
my_geopolygon = geom_fromdrawn(selected_option, shapefile=field_plots)[0]

print("The drawn shape was be used for : " + selected_option)
print("The geopolygon of interest is the following one; ")
my_geopolygon

#### Today's date

In this notebook, we focus on monitoring the crops in near-real time at parcel scale.

So, **let's imagine** what you would see if today was for example the 15th of May 2020 or any other date. 

Run the cell below and pick today's hypothetical date.

In [ ]:
# plot calendar and pick a date
hypothetical_today_date = calendar()

In [ ]:
# define the period of interest, i.e., beginning of the hypothetical year crop season
# to hypothetical today's date
hypothetical_year = hypothetical_today_date.value.year

start_date = str(hypothetical_year-1)+'-10-01'
end_date = hypothetical_today_date.value.strftime("%Y-%m-%d")

print("Analysing " + str(hypothetical_year) + " crop season: "+ start_date + " to " + end_date)

### Loading data from datacube 

#### Prepare query for datacube

In [ ]:
query = {'product': 'sen1_rtc_pyroSNAP',
             'geopolygon': my_geopolygon,
             'time': (start_date, end_date),
             'output_crs': 'epsg:27700',
             'measurements': ['VH','VV'],
             'resolution': (-10,10)}

#### Load data

In [ ]:
start_time = time()

# Let's load the dataset
dataset_in = dc.load(**query)

# define nodata
dataset_in = dataset_in.where(dataset_in != 0)

# clean data: drop dates with only nodata for the area of interest
dataset_in = dataset_in.dropna('time', how='all')

# When using epsg other than 4326 (here: 27700), latitude and longitude are renamed y and x.
# Let's correct that and rename x and y with explicit names
dataset_in = dataset_in.rename({'x': 'longitude', 'y': 'latitude'})

# group images by YYYY-MM-DD
S1_dataset = dataset_in.groupby(dataset_in.time.dt.strftime("%Y-%m-%d")).mean("time")
S1_dataset = S1_dataset.rename({'strftime': 'time'})

print("Datacube ready")
print("Took only " + str(round(time()-start_time,2)) + " seconds to load "+ str(len(S1_dataset.time
                                                                              )) +" images from datacube.")

In [ ]:
# Visualise VH backscatter for the first date in the dataset
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds.)")

S1_dataset.VH.isel(time=0).plot();

### Masking data out of the field plot

As you can see on the above figure, within the data cube, data are loaded as a geobox (i.e., a rectangle). 
In our case, in cell 9, we have queried a geopolygon. However, the data cube only uses the maximum/minimum latitude/longitude of the polygon to load the data.

In this section, we will mask out the pixels which are not included within the draw/selected geopolygon using one of the tools developed by Living Wales.

In [ ]:
# Masking data out of the field plot
S1_dataset = geopolygon_masking(S1_dataset, my_geopolygon)

# Visualise VH backscatter for the first date in the dataset (after masking)
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds.)")

S1_dataset.VH.isel(time=0).plot();

### Tracking the evolution of a parcel in near-real time

As with Sentinel-2, indices can be calculated using Sentinel-1. The polarization ratio, which is very useful to reduce the effect of soil on VH backscattering and detect the beginning of vegetation growing, is one of the most used indices when analysing radar data.

In this section, for each available date, from the beginning of the crop season (i.e., 1st October) to today's hypothetical date, we calculate the index, summarise the backscatter/index at parcel scale, and plot its evolution.

In [ ]:
# Calculate VH/VV ratio for each pixel
S1_dataset['Ratio'] = S1_dataset.VH - S1_dataset.VV

# Summarise the backscatters/indices at parcel scale
median_VH = S1_dataset.VH.median(['latitude','longitude'])
median_VV = S1_dataset.VV.median(['latitude','longitude'])
median_Ratio = S1_dataset.Ratio.median(['latitude','longitude'])

In [ ]:
# Plotting the parcel backscatters/index
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds depending on your period of interest.)")

# define figure size
fig, (ax1, ax2, ax3) = plt.subplots(3,figsize=(18,18));
# Set title
fig.suptitle('Evolution of the parcel since the beginning of the crop season', y=0.9);
# Set date format for the x axis
dates=[dt.datetime.strptime(d,'%Y-%m-%d').date() for d in median_VV.time.values];

# Plot the polarization ratio
ax1.plot(dates, median_Ratio.values, marker="o");
ax1.set_ylim([-16, -2]);
ax1.grid(visible=True, which='major', axis='both');
ax1.xaxis.set_major_locator(mdates.MonthLocator());
ax1.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"));
ax1.set_ylabel("Ratio");

# Plot the VH backscatter
ax2.plot(dates,median_VH.values, marker="o");
ax2.set_ylim([-26, -5]);
ax2.grid(visible=True, which='major', axis='both');
ax2.xaxis.set_major_locator(mdates.MonthLocator());
ax2.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"));
ax2.set_ylabel("VH backscatter");

# Plot the VV backscatter
ax3.plot(dates, median_VV.values, marker="o");
ax3.set_ylim([-20, -2]);
ax3.grid(visible=True, which='major', axis='both');
ax3.xaxis.set_major_locator(mdates.MonthLocator());
ax3.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"));
ax3.set_ylabel("VV backscatter");


--------------------------------------------------------------------------------------------------------------------



### Study case: tracking the growth stages in a Rapeseed plot

In this section, we showcase how, using Sentinel-1 data and derived index, we can track the seasonal evolution of a  plot and the dates of some key growth stages. 

Here, we will study the case of a rapeseed plot during the 2018 crop season, but all crops have their own SAR-C signature which can be related to change in growth stages.

To help you to visually interpret the SAR C-band trends, we added some cloud-free Planet imagery.


#### Load data from datacube

In [ ]:
# Use the rapeseed study plot as geopolygon
rapeseed = field_plots[field_plots.id==1]
rapeseed_plot = Geometry(geom= rapeseed.geometry[0], crs=CRS("epsg:4326"))

# Prepare query for datacube
query = {'product': 'sen1_rtc_pyroSNAP',
             'geopolygon': rapeseed_plot,
             'time': ('2017-08-01', '2018-10-31'),
             'output_crs': 'epsg:27700',
             'measurements': ['VH','VV'],
             'resolution': (-10,10)}

# Load the dataset for the plot of interest
dataset_rapeseed = dc.load(**query)
# define nodata
dataset_rapeseed = dataset_rapeseed.where(dataset_rapeseed != 0)
# clean data: drop dates with only nodata for the area of interest
dataset_rapeseed = dataset_rapeseed.dropna('time', how='all')
# rename x and y with explicit names
dataset_rapeseed = dataset_rapeseed.rename({'x': 'longitude', 'y': 'latitude'})
# group images by YYYY-MM-DD
S1_rapeseed = dataset_rapeseed.groupby(dataset_rapeseed.time.dt.strftime("%Y-%m-%d")).mean("time")
S1_rapeseed = S1_rapeseed.rename({'strftime': 'time'})
# masking rapeseed plot
S1_rapeseed = geopolygon_masking(S1_rapeseed, rapeseed_plot)

#### Calculate the polarization ratio

In [ ]:
# Calculate polarization ratio for each pixel
S1_rapeseed['Ratio'] = S1_rapeseed.VH - S1_rapeseed.VV

# Summarise the backscatters/index at parcel level
median_VH_rapeseed = S1_rapeseed.VH.median(['latitude','longitude'])
median_VV_rapeseed = S1_rapeseed.VV.median(['latitude','longitude'])
median_Ratio_rapeseed = S1_rapeseed.Ratio.median(['latitude','longitude'])

#### Tracking growth stages in the rapeseed plot

In this section, we plot the SAR C-band signal, and derived index, during the whole crop season.

On the SAR trend figure, we have indicated each Planet image available below using a vertical red line (except for 5th May image (i.e., full flowering) which is indicated in yellow).



As we can see on the figure hereafter:

- vegetation emerged around the 2017-09-15 
- followed by an active leaf production until ~ 2017-10-20 
- biomass production then slowed down until ~ 2018-02-01, and then resumed.
- biomass production accelerated from ~ 2018-04-01 (i.e., stem elongation/flower bud development)
- flowering started around 2018-04-25 
- pods/seeds started to develop around 2018-05-10
- pods fully developed by 2018-05-25, then the ripening/senescence started.
- harvest on 2018-07-07.

All the above-mentioned dates are indicated with vertical black lines on the SAR trend figure.

In [ ]:
# Plot SAR-C signal and derived index of the study plot (with some Planet images)
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds.)")

rapeseed_study_case_plot(median_VH=median_VH_rapeseed, median_VV=median_VV_rapeseed, 
                         median_Ratio=median_Ratio_rapeseed)